# Multi-label COCO

In [ ]:
#@title

! pip -qq install tensorflow-addons
! pip install -qq -U tensorflow_datasets

IS_COLAB = 'google.colab' in str(get_ipython())

if IS_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

import tensorflow as tf

for d in tf.config.list_physical_devices('GPU'):
  print(d)
  print(f'Setting device {d} to memory-growth mode.')
  try:
    tf.config.experimental.set_memory_growth(d, True)
  except Exception as e:
    print(e)

In [ ]:
if IS_COLAB:
  base_dir = '/content/drive/MyDrive'
  data_dir = '/root/tensorflow_datasets'
else:
  base_dir = '/home/ldavid/Workspace'
  data_dir = '/home/ldavid/Workspace/datasets'


class Config:
  class data:
    data_dir = '/home/ldavid/Workspace/datasets'
    size = (512, 512)
    shape = (*size, 3)
    batch_size = 32
    shuffle_buffer_size = 8 * batch_size
    prefetch_buffer_size = tf.data.experimental.AUTOTUNE
    train_shuffle_seed = 2142
    shuffle = True

    preprocess = tf.keras.applications.resnet_v2.preprocess_input
    deprocess = lambda x: tf.cast(tf.clip_by_value((x+1)*127.5, 0, 255), tf.uint8)

  class aug:
    brightness_delta =  .2
    saturation_lower =  .2
    saturation_upper = 1.0
    contrast_lower   =  .5
    contrast_upper   = 1.5
    hue_delta        =  .0
    
  class model:
    pool_layer = 'avg_pool'
    backbone = tf.keras.applications.ResNet101V2

    ae_latent_dim = 128
    ae_intermediate_dim = 512
  
  class training:
    epochs = 80
    learning_rate = 1.0
    lr_first_decay_steps = 50

    fine_tune_lr = 0.01
    fine_tune_epochs = 80
    fine_tune_layers = .6
    freeze_batch_norm = False

    fine_tune_early_reduce_lr_patience = 3
    fine_tune_early_stopping_patience = 20

  class experiment:
    seed = 218402
    override = True
    logs              = f'{base_dir}/logs/coco/resnet101-sw-1-ce/'
    fine_tune_logs    = f'{base_dir}/logs/coco/resnet101-sw-1-ce-fine-tune/'

    training_weights  = f'{base_dir}/models/coco/resnet101-sw-1-ce/weights.h5'
    fine_tune_weights = f'{base_dir}/models/coco/resnet101-sw-1-ce-fine-tune/weights.h5'

## Setup

In [ ]:
import os
import shutil
from math import ceil

import numpy as np
import pandas as pd
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import callbacks

In [ ]:
for d in tf.config.list_physical_devices('GPU'):
  print(d)
  print(f'Setting device {d} to memory-growth mode.')
  try:
    tf.config.experimental.set_memory_growth(d, True)
  except Exception as e:
    print(e)

In [ ]:
R = tf.random.Generator.from_seed(Config.experiment.seed, alg='philox')
C = np.asarray(sns.color_palette("Set1", 21))
CMAP = sns.color_palette("Set1", 21, as_cmap=True)

sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
def normalize(x, reduce_min=True, reduce_max=True):
  if reduce_min: x -= tf.reduce_min(x, axis=(-3, -2), keepdims=True)
  if reduce_max: x = tf.math.divide_no_nan(x, tf.reduce_max(x, axis=(-3, -2), keepdims=True))

  return x


def visualize(
    image,
    title=None,
    rows=2,
    cols=None,
    figsize=(16, 7.2),
    cmap=None
):
  if image is not None:
    if isinstance(image, (list, tuple)) or len(image.shape) > 3:  # many images
      plt.figure(figsize=figsize)
      cols = cols or ceil(len(image) / rows)
      for ix in range(len(image)):
        plt.subplot(rows, cols, ix+1)
        visualize(image[ix],
                 cmap=cmap,
                 title=title[ix] if title is not None and len(title) > ix else None)
      plt.tight_layout()
      return

    if isinstance(image, tf.Tensor): image = image.numpy()
    if image.shape[-1] == 1: image = image[..., 0]
    plt.imshow(image, cmap=cmap)
  
  if title is not None: plt.title(title)
  plt.axis('off')

## Dataset

In [ ]:
%%bash

if [ ! -d "/root/tensorflow_datasets/coco" ]; then
  mkdir -p /root/tensorflow_datasets/coco/2017/1.1.0

  cp /content/drive/MyDrive/datasets/coco/2017/1.1.0/*train* \
     /content/drive/MyDrive/datasets/coco/2017/1.1.0/*validation* \
     /content/drive/MyDrive/datasets/coco/2017/1.1.0/*.json       \
     /content/drive/MyDrive/datasets/coco/2017/1.1.0/*.txt        \
     /root/tensorflow_datasets/coco/2017/1.1.0
else
  echo "Data found at coco/2017 - skipping"
fi

In [ ]:
def default_policy_fn(image):
  image = tf.image.resize_with_crop_or_pad(image, *Config.data.size)
  # mask = tf.image.resize_with_crop_or_pad(mask, *Config.data.size)

  return image


def augment_policy_fn(image):
  seeds = R.make_seeds(6)

  image = tf.image.resize_with_crop_or_pad(image, *Config.data.size)
  # image = tf.image.stateless_random_crop(image, [*Config.data.size, 3], seed=seeds[:, 0])
  # mask = tf.image.stateless_random_crop(mask, [*Config.data.size, 1], seed=seeds[:, 0])

  image = tf.image.stateless_random_flip_left_right(image, seed=seeds[:, 0])
  # mask = tf.image.stateless_random_flip_left_right(mask, seed=seeds[:, 0])
  
  image = tf.image.stateless_random_flip_up_down(image, seed=seeds[:, 1])
  # mask = tf.image.stateless_random_flip_up_down(mask, seed=seeds[:, 1])

  image = tf.image.stateless_random_hue(image, Config.aug.hue_delta, seed=seeds[:, 2])
  image = tf.image.stateless_random_brightness(image, Config.aug.brightness_delta, seed=seeds[:, 3])
  image = tf.image.stateless_random_contrast(image, Config.aug.contrast_lower, Config.aug.contrast_upper, seed=seeds[:, 4])
  image = tf.image.stateless_random_saturation(image, Config.aug.saturation_lower, Config.aug.saturation_upper, seed=seeds[:, 5])

  return image

In [ ]:
(train_dataset, val_dataset), info = tfds.load(
  'coco/2017',
  split=('train', 'validation'),
  with_info=True,
  shuffle_files=False,
)

In [ ]:
CLASSES = np.asarray(info.features['objects']['label']._int2str)
int2str = info.features['objects']['label'].int2str

In [ ]:
from functools import partial


@tf.function
def load_fn(d, augment=False):
  image = d['image']
  labels = d['objects']['label']

  image = tf.cast(image, tf.float32)
  
  image, _ = adjust_resolution(image)
  image = (augment_policy_fn(image)
           if augment
           else default_policy_fn(image))
  
  image = Config.data.preprocess(image)

  return image, labels_to_one_hot(labels)


def adjust_resolution(image):
  es = tf.constant(Config.data.size, tf.float32)
  xs = tf.cast(tf.shape(image)[:2], tf.float32)

  ratio = tf.reduce_min(es / xs)
  xsn = tf.cast(tf.math.ceil(ratio * xs), tf.int32)

  image = tf.image.resize(image, xsn, preserve_aspect_ratio=True, method='nearest')

  return image, ratio


def labels_to_one_hot(labels):
  return tf.maximum(tf.reduce_max(tf.one_hot(labels, depth=CLASSES.shape[0]), axis=0), 0)


def prepare(ds, batch_size, cache=False, shuffle=False, augment=False):
  if cache: ds = ds.cache()
  if shuffle: ds = ds.shuffle(Config.data.shuffle_buffer_size, reshuffle_each_iteration=True, seed=Config.data.train_shuffle_seed)

  return (ds.map(partial(load_fn, augment=augment), num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size, drop_remainder=True)
            .prefetch(Config.data.prefetch_buffer_size))

## Network

In [ ]:
print(f'Loading {Config.model.backbone.__name__}')

backbone = Config.model.backbone(
  classifier_activation=None,
  include_top=False,
  input_shape=Config.data.shape
)

backbone.trainable = False

### Softmax weights

Dense With Softmax Weights Layer:

\begin{align}
l_c &= X \cdot [W \circ \text{softmax}_c(W - max_c(W))] + b\\
y_c &= \sigma(l)_c
\end{align}

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense


class DenseKur(Dense):
  """Dense with Softmax Weights.
  """
  def call(self, inputs):
    kernel = self.kernel
    ag = kernel  # ag = tf.abs(kernel)
    ag = ag - tf.reduce_max(ag, axis=-1, keepdims=True)
    ag = tf.nn.softmax(ag)

    outputs = inputs @ (ag * kernel)

    if self.use_bias:
      outputs = tf.nn.bias_add(outputs, self.bias)

    if self.activation is not None:
      outputs = self.activation(outputs)

    return outputs

### Classification Head

In [ ]:
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense


def build_classifier():
  x = tf.keras.Input(Config.data.shape, name='images')
  y = backbone(x)
  y = GlobalAveragePooling2D(name='avg_pool')(y)
  y = DenseKur(len(CLASSES), name='predictions')(y)

  return tf.keras.Model(
    inputs=x,
    outputs=y,
    name=f'clf_{Config.model.backbone.__name__}_coco')

In [ ]:
nn = build_classifier()

In [ ]:
tf.keras.utils.plot_model(nn, show_shapes=True)

## Training

### Loss, Metrics and Model Compilation

In [ ]:
class FromLogitsMixin:
  def __init__(self, from_logits=False, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.from_logits = from_logits

  def update_state(self, y_true, y_pred, sample_weight=None):
    if self.from_logits:
      y_pred = tf.nn.sigmoid(y_pred)
    return super().update_state(y_true, y_pred, sample_weight)


class BinaryAccuracy(FromLogitsMixin, tf.metrics.BinaryAccuracy):
  ...

class TruePositives(FromLogitsMixin, tf.metrics.TruePositives):
  ...

class FalsePositives(FromLogitsMixin, tf.metrics.FalsePositives):
  ...

class TrueNegatives(FromLogitsMixin, tf.metrics.TrueNegatives):
  ...

class FalseNegatives(FromLogitsMixin, tf.metrics.FalseNegatives):
  ...

class Precision(FromLogitsMixin, tf.metrics.Precision):
  ...

class Recall(FromLogitsMixin, tf.metrics.Recall):
  ...

class F1Score(FromLogitsMixin, tfa.metrics.F1Score):
  ...

In [ ]:
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.platform import tf_logging

class ReduceLRBacktrack(ReduceLROnPlateau):
    """
    Reduce Learning Rate on Plateau and restore weights.
    
    Ref: https://stackoverflow.com/a/55228619/2429640
    """
    def __init__(self, best_path, *args, **kwargs):
        super(ReduceLRBacktrack, self).__init__(*args, **kwargs)
        self.best_path = best_path

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            tf_logging.warning('Reduce LR on plateau conditioned on metric `%s` '
                               'which is not available. Available metrics are: %s',
                               self.monitor, ','.join(list(logs.keys())))
        if not self.monitor_op(current, self.best):
            if not self.in_cooldown():
                if self.wait+1 >= self.patience:
                    print("Backtracking to best model before reducting LR")
                    self.model.load_weights(self.best_path)

        super().on_epoch_end(epoch, logs)

### Top Classifier Training

In [ ]:
train = prepare(train_dataset, 2*Config.data.batch_size, shuffle=True, augment=True)
valid = prepare(val_dataset, 2*Config.data.batch_size)

In [ ]:
nn.compile(
    optimizer=tf.optimizers.SGD(learning_rate=Config.training.learning_rate, momentum=0.9, nesterov=True),
    loss=tf.losses.BinaryCrossentropy(from_logits=True),
    metrics=[
      F1Score(num_classes=len(CLASSES), from_logits=True, average='macro'),
      Precision(from_logits=True),
      Recall(from_logits=True),
      tf.keras.metrics.AUC(multi_label=True, from_logits=True),
    ])

In [ ]:
def train_fn(initial_epoch):
  try:
    os.makedirs(os.path.dirname(Config.experiment.training_weights), exist_ok=True)

    if os.path.exists(Config.experiment.logs) and initial_epoch == 0:
      if not Config.experiment.override:
        raise ValueError(f'A training was found in {Config.experiment.logs}. '
                        f'Either move it or set experiment.override to True.')

      print(f'Overriding previous training at {Config.experiment.logs}.')
      shutil.rmtree(Config.experiment.logs)

    nn.fit(
      train,
      validation_data=valid,
      epochs=Config.training.epochs,
      initial_epoch=initial_epoch,
      callbacks=[
        callbacks.TerminateOnNaN(),
        callbacks.ModelCheckpoint(Config.experiment.training_weights, save_best_only=True, save_weights_only=True, verbose=1),
        callbacks.TensorBoard(Config.experiment.logs, profile_batch=False),
        callbacks.EarlyStopping(patience=20, verbose=1),
        ReduceLRBacktrack(Config.experiment.training_weights, factor=0.5, patience=3, min_lr=0.001, verbose=1),
      ]);

  except KeyboardInterrupt: print('\ninterrupted')
  else: print('\ndone')

In [ ]:
train_fn(initial_epoch=0)

In [ ]:
nn.load_weights(Config.experiment.training_weights)

train_fn(initial_epoch=21)

### Fine-Tuning

In [ ]:
train = prepare(train_dataset, Config.data.batch_size, shuffle=True, augment=True)
valid = prepare(val_dataset, Config.data.batch_size)

In [ ]:
nn.load_weights(Config.experiment.training_weights)

In [ ]:
if Config.training.fine_tune_epochs:
  backbone.trainable = True

  frozen_layer_ix = int((1-Config.training.fine_tune_layers) * len(backbone.layers))
  for ix, l in enumerate(backbone.layers):
    l.trainable = (ix > frozen_layer_ix and
                   (not isinstance(l, tf.keras.layers.BatchNormalization) or
                    not Config.training.freeze_batch_norm))

  nn.compile(
    optimizer=tf.optimizers.SGD(learning_rate=Config.training.fine_tune_lr,
                                momentum=0.9,
                                nesterov=True),
    loss=tf.losses.BinaryCrossentropy(from_logits=True),
    metrics=[
      F1Score(num_classes=len(CLASSES), from_logits=True, average='macro'),
      Precision(from_logits=True),
      Recall(from_logits=True),
      tf.keras.metrics.AUC(multi_label=True, from_logits=True),
    ])

In [ ]:
trained_epochs = 28

In [ ]:
if Config.training.fine_tune_epochs:
  os.makedirs(os.path.dirname(Config.experiment.fine_tune_weights), exist_ok=True)

  print(f'Fine tuning params:')
  print(f'  epochs:          {Config.training.fine_tune_epochs}')
  print(f'  learning rate:   {Config.training.fine_tune_lr}')
  print(f'  layers unfrozen: {frozen_layer_ix} to {len(backbone.layers)}')

  try:
    history = nn.fit(
      train,
      validation_data=valid,
      epochs=trained_epochs + Config.training.fine_tune_epochs,
      callbacks=[
        callbacks.TerminateOnNaN(),
        callbacks.ModelCheckpoint(Config.experiment.fine_tune_weights, save_best_only=True, save_weights_only=True, verbose=1),
        callbacks.TensorBoard(Config.experiment.fine_tune_logs),
        callbacks.EarlyStopping(patience=20, verbose=1),
        ReduceLRBacktrack(Config.experiment.fine_tune_weights, factor=0.5, patience=3, min_lr=0.001, verbose=1),
      ],
      initial_epoch=trained_epochs,
    );

  except KeyboardInterrupt: print('\ninterrupted')
  else: print('\ndone')

## Evaluation

In [ ]:
#@title


def labels_and_probs(nn, dataset):
  labels_ = []
  probs_ = []

  for images, labels in dataset:
    y = nn(images, training=False)
    y = tf.nn.sigmoid(y)

    labels_.append(labels)
    probs_.append(y)

    print('.', end='')
  
  return (tf.concat(labels_, axis=0),
          tf.concat(probs_, axis=0))


def evaluate(l, p):
  acc, tpr, fpr, tnr, fnr, auc, mcm = metrics_per_label(l, p)

  return pd.DataFrame({
    'accuracy': acc,
    'true positive r': tpr,
    'true negative r': tnr,
    'false positive r': fpr,
    'false negative r': fnr,
    'roc auc score': auc,
    'support': tf.cast(tf.reduce_sum(l, axis=0), tf.int32),
    'label': CLASSES
  })

In [ ]:
if Config.training.fine_tune_epochs:
  backbone.trainable = True

  frozen_layer_ix = int((1-Config.training.fine_tune_layers) * len(backbone.layers))

  for ix, l in enumerate(backbone.layers):
    l.trainable = (ix > frozen_layer_ix and
                   (not isinstance(l, tf.keras.layers.BatchNormalization) or
                    not Config.training.freeze_batch_norm))

In [ ]:
nn.load_weights('/home/ldavid/Workspace/models/coco/resnet101-ce-fine-tune/weights.h5')

In [ ]:
l, p = labels_and_probs(nn, valid)

In [ ]:
from sklearn import metrics as skmetrics

def metrics_per_label(gt, probs, threshold=0.5):
  p_pred = tf.cast(probs > threshold, probs.dtype).numpy()

  tru_ = tf.reduce_sum(gt, axis=0)
  neg_ = tf.reduce_sum(1- gt, axis=0)

  tpr = tf.reduce_sum(p_pred*gt, axis=0) / tru_
  fpr = tf.reduce_sum(p_pred*(1-gt), axis=0) / neg_
  tnr = tf.reduce_sum((1-p_pred)*(1-gt), axis=0) / neg_
  fnr = tf.reduce_sum((1-p_pred)*gt, axis=0) / tru_

  f2_score = skmetrics.fbeta_score(gt, p_pred, beta=2, average=None)
  precision, recall, f1_score, support = skmetrics.precision_recall_fscore_support(
    gt, p_pred, average=None)

  mcm = skmetrics.multilabel_confusion_matrix(gt, p_pred)

  skmetrics.accuracy_score(gt, p_pred)

  return pd.DataFrame({
    'true positive r': tpr,
    'true negative r': tnr,
    'false positive r': fpr,
    'false negative r': fnr,
    'precision': precision,
    'recall': recall,
    'auc_score': skmetrics.roc_auc_score(gt, probs, average=None),
    'f1_score': f1_score,
    'f2_score': f2_score,
    'support': support,
    'label': CLASSES
  }), mcm

test_report, test_mcm = metrics_per_label(l, p)

In [ ]:
with pd.option_context('display.max_rows', 100):
  display(test_report.round(4))

In [ ]:
co_occurrence = tf.transpose(l) @ l
occurrence = tf.reshape(np.diag(co_occurrence), (-1, 1))

co_occurrence_rate = tf.math.divide_no_nan(co_occurrence, occurrence)

In [ ]:
#@title Labels Occurrence Matrix

plt.figure(figsize=(16, 12))
sns.heatmap(
  co_occurrence_rate.numpy(),
  annot=False,
  fmt='.0%',
  xticklabels=CLASSES,
  yticklabels=CLASSES,
  cmap="RdPu",
  cbar=False
)
plt.tight_layout();

In [ ]:
d = tf.cast(p > 0.5, tf.float32)
co_occurrence = tf.transpose(d) @ d
occurrence = tf.reshape(np.diag(co_occurrence), (-1, 1))

co_occurrence_rate = tf.math.divide_no_nan(co_occurrence, occurrence)

In [ ]:
#@title Predictions Relation Matrix

plt.figure(figsize=(16, 12))
sns.heatmap(
  co_occurrence_rate.numpy(),
  annot=False,
  fmt='.0%',
  xticklabels=CLASSES,
  yticklabels=CLASSES,
  cmap="RdPu",
  cbar=False
)
plt.tight_layout();